## Documentation

To read more about the dense vector field type, visit the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Inserting documents

Because we are using the `dense_vector` data type, we need to do the mapping manually.

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties": {
            "sides_length": {
                "type": "dense_vector",
                "dims": 4
            },
            "shape": {
                "type": "keyword"
            }
        }
    },
)

### 1. Valid case

We specified that `sides_length` should be a vector with 4 values. Since this condition was met, the operation was executed successfully.

In [ ]:
from pprint import pprint

response = es.index(
    index='my_index',
    id=1,
    document={
        "shape": "square",
        "sides_length": [5, 5, 5, 5],
    }
)

pprint(response.body)

Let's check the mapping. We can see that `sides_length` is a `dense_vector` with 4 dimensions and `cosine` set as the similarity measure. These are parameters we can modify if needed. Since we didn’t specify them when manually setting the mapping, Elasticsearch automatically applied the default values.

In [ ]:
pprint(es.indices.get_mapping(index='my_index').body)

### 2. Invalid case

In this case, the operation fails because indexing a matrix is not supported with the `dense_vector` field type.

In [ ]:
response = es.index(
    index='my_index',
    id=2,
    document={
        "shape": "square",
        "sides_length": [[5, 5], [5, 5]],
    }
)

pprint(response.body)